In [1]:
from autocode import Optimization

optimization: Optimization = Optimization(
    num_cpus=16,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

2024-08-06 00:01:57,649	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 16.0, 'memory': 8791965696.0, 'object_store_memory': 4395982848.0, 'GPU': 1.0}
Server is healthy.


In [3]:
from autocode import OptimizationUseCase, ApplicationSetting, OneDatastore
from autocode.container import ApplicationContainer
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "autocode"

application_container: ApplicationContainer = ApplicationContainer()
application_setting: ApplicationSetting = application_container.settings.application()
application_setting.num_cpus = 16
application_setting.server_host = "0.0.0.0"
application_setting.server_port = 10000
application_setting.dashboard_port = 10001
optimization_use_case: OptimizationUseCase = application_container.use_cases.optimization()
# optimization_use_case.reset()

In [4]:
from python_on_whales import DockerClient

docker_client: DockerClient = optimization.deploy(
    compose_files=["./client-compose.yml"],
)

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 1.9s

#4 [app-account internal] load .dockerignore
#4 transferring context: 2B 0.0s done
#4 DONE 0.0s

#5 [app-product 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#5 DONE 0.0s

#6 [app-product internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.0s

#7 [app-product internal] load build context
#7 transferring context: 335B 0.1s done
#7 DONE 0.1s

#8 [app-account internal] load build context
#8 transferring context: 335B 0.1s done
#8 DONE 0.1s

#9 [app-product 3/5] COPY . .
#9 CACHED

#10 [app-product 4/5] RUN go mod

 Container example-app-product-1  Created
 Container example-app-product-2  Created
 Container example-app-product-3  Created
 Container example-app-product-4  Created
 Container example-app-product-5  Created
 Container example-app-product-6  Created
 Container example-app-product-7  Created
 Container example-app-product-8  Created
 Container example-app-product-9  Created
 Container example-app-product-10  Created
 Container example-app-account-1  Created
 Container example-app-product-11  Created
 Container example-app-account-2  Created
 Container example-app-product-12  Created
 Container example-app-account-3  Created
 Container example-app-product-13  Created
 Container example-app-account-4  Created
 Container example-app-product-14  Created
 Container example-app-account-5  Created
 Container example-app-product-15  Created
 Container example-app-account-6  Created
 Container example-app-product-16  Created
 Container example-app-account-7  Created
 Container example-app-acco

In [5]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(inputs: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    output: Dict[str, Any] = {
        "F": inputs[0].objectives + inputs[1].objectives,
    }

    return output


In [6]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     27 |             - |             -
     2 |      200 |     38 |  0.1805555556 |         nadir
     3 |      300 |     34 |  1.8734177215 |         nadir
     4 |      400 |     36 |  0.0500000000 |         ideal
     5 |      500 |     39 |  0.0595238095 |         ideal
     6 |      600 |     42 |  0.0088675221 |             f
     7 |      700 |     47 |  0.0232558140 |         ideal
     8 |      800 |     48 |  0.0116746831 |             f
     9 |      900 |     47 |  0.0046415783 |             f
    10 |     1000 |     44 |  0.0172413793 |         nadir
    11 |     1100 |     48 |  0.0169491525 |         nadir
    12 |     1200 |     45 |  0.0187501555 |             f
    13 |     1300 |     43 |  0.0075188311 |             f
    14 |     1400 |     44 |  0.0006540191 |             f
    15 |     1500 |     45 |  0.0072060318 |             f
    16 |     1600 |     45 |  0.0131238633 |            